In [61]:
from urllib.request import urlopen
import certifi
import json
from datetime import datetime
import calendar
from functools import reduce
from operator import mul

import pandas as pd
import yfinance as yf

In [2]:
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
API_KEY = "e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

FETCH_STOCKS_URL = (
    f"https://financialmodelingprep.com/api/v3/stock-screener?apikey={API_KEY}&"
    f"exchange=nyse,nasdaq&isEtf=false&isFund=false&limit=10000"
)
RATIO_URL = f"https://financialmodelingprep.com/api/v3/ratios/%s?period=quarter&apikey={API_KEY}"
EARNING_URL = f"https://financialmodelingprep.com/api/v3/earnings-surprises/%s?apikey={API_KEY}"
HISTORICAL_PRICE_URL = f"https://financialmodelingprep.com/api/v3/historical-price-full/%s?apikey={API_KEY}&from=%s&to=%s"

In [4]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [5]:
def corps_total_rank_by_standard_columns(df, column_map) -> pd.DataFrame:
    report_df = df.copy()
    rank_df = pd.DataFrame(index=report_df.index)
    total_value = pd.Series([0] * len(report_df), index=report_df.index)

    for col, ascending in column_map.items():
        total_value += report_df[col].rank(ascending=ascending)

    rank_df['total_rank'] = total_value.rank()
    rank_df = rank_df.sort_values(by='total_rank')
    report_df['rank'] = rank_df['total_rank']

    return report_df.loc[rank_df.index]

In [83]:
stocks = get_jsonparsed_data(FETCH_STOCKS_URL)
len(stocks)

7856

In [84]:
tickers = list(map(lambda stock: stock["symbol"], stocks))
tickers = sorted(tickers)

In [106]:
metrics = []

for idx, ticker in enumerate(tickers):
    try:
        earnings = get_jsonparsed_data(EARNING_URL % ticker)
        price_chart = None
        if len(earnings) > 0:
            start_date = earnings[-1]['date']
            end_date = earnings[0]['date']
            price_chart = yf.Ticker(ticker).history(start=start_date, end=end_date)
            print(f"======= {ticker} {start_date} ~ {end_date} {idx + 1} / {len(tickers)} ========")
        else:
            print(ticker, "not exist")
        for earning in earnings:
            date_str = earning["date"]
            date = datetime.strptime(date_str, "%Y-%m-%d")
            estimatedEarning = earning['estimatedEarning']
            price = 0
            try:
                price = price_chart.loc[f"{date.year}-{date.month}"]['Close'][-1]
            except:
                None
            metrics.append({
                'symbol' : earning['symbol'],
                'date' : date,
                'year' : date.year,
                'month' : date.month,
                'day' : date.day,
                'price' : price,
                'actual' : earning['actualEarningResult'],
                'estimated' : estimatedEarning,
                'earning_rate' : 1 + (earning['actualEarningResult'] - estimatedEarning) / estimatedEarning if estimatedEarning != 0 and estimatedEarning is not None else 0
            })
        
        if (idx + 1) % 1000 == 0:
            pd.DataFrame(metrics).to_csv('earning_tmp.csv', index = False)
    except Exception as e:
        print(e)
        None

======= A 2000-02-17 ~ 2024-11-25 1 / 7856 ========
======= AA 2017-01-24 ~ 2024-10-16 2 / 7856 ========
AAC not exist
======= AACG 2008-05-15 ~ 2023-05-11 4 / 7856 ========
AACI not exist
AACT not exist
======= AADI 2018-05-15 ~ 2024-11-06 7 / 7856 ========
AAGR not exist


AAIC: No timezone found, symbol may be delisted


======= AAIC 1998-04-23 ~ 2023-08-14 9 / 7856 ========
AAIN not exist
======= AAL 1993-01-20 ~ 2024-10-24 11 / 7856 ========
AAME not exist


AAN: No timezone found, symbol may be delisted


======= AAN 1993-01-27 ~ 2024-10-28 13 / 7856 ========
======= AAOI 2014-02-27 ~ 2024-11-07 14 / 7856 ========
======= AAON 1994-07-12 ~ 2024-11-07 15 / 7856 ========
======= AAP 2002-02-20 ~ 2024-11-14 16 / 7856 ========
======= AAPL 1995-07-20 ~ 2024-10-31 17 / 7856 ========
AAQC not exist
======= AAT 2011-08-09 ~ 2024-10-29 19 / 7856 ========


...

======= ZVO 2009-08-11 ~ 2022-04-15 7847 / 7856 ========
ZVOL not exist
======= ZVRA 2015-05-27 ~ 2024-08-13 7849 / 7856 ========
======= ZVSA 2023-11-14 ~ 2024-11-14 7850 / 7856 ========
ZWRK not exist
======= ZWS 2012-08-01 ~ 2024-10-29 7852 / 7856 ========


ZY: No timezone found, symbol may be delisted


======= ZY 2021-08-16 ~ 2022-08-15 7853 / 7856 ========
======= ZYME 2017-08-08 ~ 2024-10-31 7854 / 7856 ========


ZYNE: No timezone found, symbol may be delisted


======= ZYNE 2015-11-11 ~ 2023-11-13 7855 / 7856 ========
======= ZYXI 2011-05-05 ~ 2024-10-24 7856 / 7856 ========


In [108]:
stocks_df = pd.DataFrame(metrics)

In [109]:
stocks_df

,symbol,date,year,month,day,price,actual,estimated,earning_rate
0,A,2024-11-25,2024,11,25,133.839996,1.46000,1.41000,1.035461
1,A,2024-08-21,2024,8,21,142.692841,1.32000,1.26000,1.047619
2,A,2024-05-29,2024,5,29,129.961533,1.22000,1.19000,1.025210
3,A,2024-02-27,2024,2,27,136.665619,1.29000,1.22000,1.057377
4,A,2023-11-20,2023,11,20,126.939247,1.38000,1.34000,1.029851
...,...,...,...,...,...,...,...,...,...
284447,ZYXI,2013-05-08,2013,5,8,0.234805,-0.01000,-0.00333,3.003003
284448,ZYXI,2013-03-14,2013,3,14,0.293506,0.01000,0.02000,0.500000
284449,ZYXI,2012-11-07,2012,11,7,0.332640,0.01000,0.02000,0.500000
284450,ZYXI,2011-08-03,2011,8,3,0.396233,0.02000,0.02000,1.000000


In [110]:
for ticker in stocks_df['symbol'].drop_duplicates():
    current_df = stocks_df[stocks_df['symbol'] == ticker]
    index = current_df[current_df['symbol'] == ticker].index
    
    prev_df = current_df.shift(-1)
    prev_prev_df = current_df.shift(-2)
    prev_prev_prev_df = current_df.shift(-3)
    
    current_price = current_df['price']
    current_actual = current_df['actual']
    prev_price = prev_df['price']
    prev_actual = prev_df['actual']
    prev_prev_actual = prev_prev_df['actual']
    prev_prev_prev_actual = prev_prev_prev_df['actual']
    
    stocks_df.loc[index, 'price_rate'] = 1 + (current_price - prev_price) / prev_price
    stocks_df.loc[index, 'earning_growth'] = 1 + (current_actual - prev_actual) / prev_actual
    stocks_df.loc[index, 'earning_growth_prev'] = 1 + (prev_actual - prev_prev_actual) / prev_prev_actual
    stocks_df.loc[index, 'earning_growth_prev_prev'] = 1 + (prev_prev_actual - prev_prev_prev_actual) / prev_prev_prev_actual
    
    stocks_df.loc[index, 'earning_rate_prev'] = 1 + (prev_actual - prev_df['estimated']) / prev_df['estimated']
    stocks_df.loc[index, 'earning_rate_prev_prev'] = 1 + (prev_prev_actual - prev_prev_df['estimated']) / prev_prev_df['estimated']
    stocks_df.loc[index, 'earning_rate_prev_prev_prev'] = 1 + (prev_prev_prev_actual - prev_prev_prev_df['estimated']) / prev_prev_prev_df['estimated']

In [136]:
years = range(2023, 2025)
months = (3, 6, 9, 12)

now = datetime.now()

total_earn_rates = []

for year in years:
    for month in months:
        if year == now.year and month >= now.month: break
            
        stock_start_date = pd.to_datetime(f"{year}-{month}-1") - pd.DateOffset(months=3)
        stock_end_date = pd.to_datetime(f"{year}-{month}-1") - pd.Timedelta(1, unit='d')
        
        quarter_stocks_df = stocks_df[
            stocks_df['date'].between(stock_start_date, stock_end_date)
        ].drop_duplicates(subset='symbol')
        earning_stocks_df = quarter_stocks_df.query(
            'price_rate > 1.1 and ' +
            'earning_rate > 1.1 and ' + 
            'earning_rate_prev > 1.1 and ' +
            'earning_rate_prev_prev > 1 and ' +
#             'earning_rate_prev_prev_prev > 1 and ' +
            'earning_growth > 1.1 and ' +
            'earning_growth_prev > 1.1 and '+
#             'earning_growth_prev_prev > 1 and '+
            'actual > 0'
        )
        selected_stocks_df = corps_total_rank_by_standard_columns(earning_stocks_df, {
            'actual': False,
            'estimated': True,
#             'earning_rate': False,
#             'earning_growth' : False,
#             'earning_rate_prev': False,
#             'earning_growth_prev': False,
#             'earning_rate_prev_prev' : False
        }).head(6)
        
        print(year, f"{month} - {month + 2}", f"from {len(earning_stocks_df)} / {len(quarter_stocks_df)}")
        
        earn_rates = []
        
        price_from_date = f"{year}-{str(month).zfill(2)}-01"
        price_to_date = pd.to_datetime(price_from_date) + pd.DateOffset(months=3) - pd.Timedelta(1, unit='d')
        price_to_date = price_to_date.strftime('%Y-%m-%d')
        
        for ticker in selected_stocks_df['symbol']:
            try:
                historical_price = pd.DataFrame(
                    get_jsonparsed_data(HISTORICAL_PRICE_URL % (ticker, price_from_date, price_to_date))['historical']
                )
                from_price = historical_price.iloc[-1]['adjClose']
                to_price = historical_price.iloc[0]['adjClose']
                
#                 print('price', 'origin', from_price, to_price)
                
#                 new_report_df = stocks_df[
#                     stocks_df['symbol'] == ticker
#                 ][stocks_df['date'].between(price_from_date, price_to_date)]
                
#                 if len(new_report_df) > 0:
#                     earning_growth = new_report_df.iloc[0]['earning_growth']
                    
#                     if earning_growth < 1:
#                         shock_date = new_report_df.iloc[0]['date'].strftime("%Y-%m-%d")
#                         to_price = historical_price[historical_price['date'] == shock_date].iloc[0]['adjClose']
#                         print('earning_shock', 'new to price', to_price)
                
                earn_rate = 1 + (to_price - from_price) / from_price
                earn_rates.append(earn_rate)
                print(ticker, from_price, to_price, earn_rate)
            except Exception as e:
                print(ticker, e)
                None
        
        earn_rate = sum(earn_rates, 0.0) / len(earn_rates)
        total_earn_rates.append(earn_rate)
        print(earn_rate)

2023 3 - 5 from 18 / 2982
H 116.37 106.77 0.9175045114720288
HEES 52.31 34.77 0.6646912636207227
THR 27.07 22.94 0.8474325821943111
LNTH 73.08 86.589996 1.1848658456486043
LW 98.2 108.51 1.104989816700611
BIVI 65.6 58.4 0.8902439024390244
0.9349546536792172
2023 6 - 8 from 12 / 4432
IDCC 81.59 85.05 1.042407157739919
ANIP 46.419998 64.39 1.387117681478573
BILL 103.989998 115.3 1.1087604790606882
AMTX 5.2 5.01 0.9634615384615384
AZUL 10.69 8.69 0.8129092609915809
DT 51.639999 48.2 0.933384990964078
1.0413401847827295
2023 9 - 11 from 42 / 4324
WIX 97.46 101.5 1.0414529037553868
MOD 48.95 49.2 1.0051072522982636
KRT 24.25 20.39 0.8408247422680413
ARRY 24.95 15.47 0.6200400801603208
POWL 83.84 82.67 0.9860448473282443
FROG 29.72 26.98 0.9078061911170929
0.9002126694878916
2023 12 - 14 from 27 / 4245
CXDO 3.3 5.55 1.6818181818181819
LOGI 85.74 86.44 1.0081642174014462
MNTX 6.9 6.84 0.9913043478260869
SURG 6.76 7.76 1.1479289940828403
NTNX 44.78 63.16 1.4104510942384993
ESTC 110.2 133.81 1.

In [137]:
reduce(mul, total_earn_rates)

1.6237190541710762

In [58]:
# 현재 요 전략이 수익률 좋음
current_stocks_df = stocks_df[
    (stocks_df['month'].isin([9, 10, 11])) & (stocks_df['year'] == 2024)
].drop_duplicates(
    subset='symbol'
).query(
    'earning_rate > 1.1 and ' + 
    'earning_rate_prev > 1.1 and ' +
    'earning_rate_prev_prev > 1.1 and ' +
    'earning_growth > 1.1 and ' +
    'earning_growth_prev > 1.1 and '+
    'actual > 0'
)
corps_total_rank_by_standard_columns(current_stocks_df, {
    'actual': False,
    'earning_growth' : False
}).head(7)

,symbol,date,year,month,day,price,actual,estimated,earning_rate,price_rate,earning_growth,earning_growth_prev,earning_growth_prev_prev,earning_rate_prev,earning_rate_prev_prev,earning_rate_prev_prev_prev,rank
27499,MCY,2024-10-29,2024,10,29,68.41,2.54,1.150,2.208696,1.169002,2.330275,1.397436,0.678261,1.362500,5.200000,23.000000,1.0
22442,ILMN,2024-11-04,2024,11,4,153.49,1.14,0.870,1.310345,1.319096,3.166667,4.000000,0.642857,3.284672,2.064220,7.860752,2.0
30326,NL,2024-11-06,2024,11,6,8.50,0.74,0.212,3.490566,1.644101,4.625000,1.142857,0.875000,1.230769,1.166667,2.666667,3.0
18860,GHC,2024-10-30,2024,10,30,841.43,17.25,15.310,1.126715,1.088216,1.358268,1.129893,1.027422,1.453089,1.254464,0.873105,4.0
20262,HAS,2024-10-24,2024,10,24,65.35,1.73,1.280,1.351562,1.085909,1.418033,2.000000,1.605263,1.564103,2.272727,0.593750,5.0
24890,KRO,2024-11-06,2024,11,6,12.90,0.62,0.340,1.823529,1.277228,3.647059,2.428571,-1.400000,1.133333,8.750000,0.714286,6.0
14318,EBR-B,2024-11-06,2024,11,6,7.22,0.55,0.150,3.666667,0.984993,3.437500,2.000000,0.615385,3.200000,1.142857,2.166667,7.0


In [107]:
pd.DataFrame(metrics)

,symbol,date,year,month,day,price,actual,estimated,earning_rate
0,A,2024-11-25,2024,11,25,133.839996,1.46000,1.41000,1.035461
1,A,2024-08-21,2024,8,21,142.692841,1.32000,1.26000,1.047619
2,A,2024-05-29,2024,5,29,129.961533,1.22000,1.19000,1.025210
3,A,2024-02-27,2024,2,27,136.665619,1.29000,1.22000,1.057377
4,A,2023-11-20,2023,11,20,126.939247,1.38000,1.34000,1.029851
...,...,...,...,...,...,...,...,...,...
284447,ZYXI,2013-05-08,2013,5,8,0.234805,-0.01000,-0.00333,3.003003
284448,ZYXI,2013-03-14,2013,3,14,0.293506,0.01000,0.02000,0.500000
284449,ZYXI,2012-11-07,2012,11,7,0.332640,0.01000,0.02000,0.500000
284450,ZYXI,2011-08-03,2011,8,3,0.396233,0.02000,0.02000,1.000000
